In [3]:
import os
import random
import numpy as np
import pandas as pd
import re

from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from imblearn.under_sampling import NeighbourhoodCleaningRule

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [4]:
#시드 고정
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED']=str(seed)
seed_everything(42)

In [6]:
#ID 열은 필요 없으므로 로드와 함께 드랍
train = pd.read_csv('./train.csv').drop('ID',axis=1) 
test = pd.read_csv('./test.csv').drop('ID',axis=1)
submission = pd.read_csv('./sample_submission.csv')

In [10]:
# 전처리
cols = ['first_party','second_party','facts']
shortword = re.compile(r'\*\b\w{1}\b') #비문자(x개)+띄어쓰기+문자하나+띄어쓰기
tokenizer = TreebankWordTokenizer() #tokenization 패키지
stopword = stopwords.words('english') #영어 stopword 로드
lemmatizer = WordNetLemmatizer() #표제어추출 패키지
# stemming(어간추출) : 문맥고려 X vs lemmatizer(표제어추출) : 문맥고려 O
# Lemmatizer 은 정확한 분석을 위해 PoS(품사) 정보를 추가로 입력받음
# 전처리 함수 1
def preprocessing(df, cols, shortword, tokenizer, stopword, lemmatizer):
    first_party_lst = []
    second_party_lst=[]
    facts_lst = []
    for col in cols:
        #좌우 공백 제거
        df[col] = df[col].str.strip()
        # 두칸 이상의 공백을 한칸으로 변경
        df[col] = df[col].str.replace('  ', ' ')
        # 소문자로 변경
        df[col] = df[col].str.lower()
        # ",", "." 제거
        df[col] = df[col].str.replace(',','')
        df[col] = df[col].str.replace('.','') #.을 모두 제거함으로써 특정 의미 반영이 힘들 수 있을 듯
        
        if col=='first_party':#원고 column에 대해 다음 적용
            for sample in df[col]:
                # 한글자 단어 제거
                sample = shortword.sub('', sample) #sample 중 한글자 단어를 ''으로 치환
                #특수문자 제거
                sample = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]","",sample) #\uAC00-\uD7A30 : 모든 한글 음절, \s : 띄어쓰기
                #tokenizer를 이용한 단어 토큰화
                token = tokenizer.tokenize(sample)
                #불용어 제거
                new_token=[]
                for tok in token:
                    if tok not in stopword:
                        #표제어 추출(명사)
                        new_token.append(lemmatizer.lemmatize(tok,'n'))
                first_party_lst.append(new_token)
                #sklearn.feature_extraction 변환을 위해 단어들을 결합?? : 문자를 숫자 벡터로 변환하기 위하여 BoW 인코딩 벡터를 만들어야 하는데 이에 사용되기 위해 각 first party들은 한 단어로 결합되어 있어야 함
            for i in range(len(first_party_lst)):
                first_party_lst[i] = ''.join(first_party_lst[i])
        elif col == 'second_party': #피고 column에 대해 다음 적용
            for sample in df[col]:
                # 한글자 단어 제거
                sample = shortword.sub('',sample)
                #특수문자 제거
                sample = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]","",sample) #\uAC00-\uD7A30 : 모든 한글 음절, \s : 띄어쓰기
                #tokenizer를 이용한 단어 토큰화
                token = tokenizer.tokenize(sample)
                #불용어 제거
                new_token=[]
                for tok in token:
                    if tok not in stopword:
                        #표제어 추출(명사)
                        new_token.append(lemmatizer.lemmatize(tok,'n'))
                second_party_lst.append(new_token)
                #sklearn.feature_extraction 변환을 위해 단어들을 결합?? : 문자를 숫자 벡터로 변환하기 위하여 BoW 인코딩 벡터를 만들어야 하는데 이에 사용되기 위해 각 first party들은 한 단어로 결합되어 있어야 함
            for i in range(len(second_party_lst)):
                second_party_lst[i] = ''.join(second_party_lst[i])
        elif col=='facts': #왜 facts column에서는 표제어 추출을 하지 않았을까??*************************
            for sample in df[col]:
                # 한글자 단어 제거
                sample = shortword.sub('', sample)
                # 특수문자 제거
                sample = re.sub(r"[^\uAC00-\uD7A30-9a-zA-Z\s]", "", sample)
                # tokenzier를 이용한 단어 토큰화
                token = tokenizer.tokenize(sample)
                # 불용어 제거
                new_token = []
                for tok in token:
                    if tok not in stopword:
                        new_token.append(tok)
                facts_lst.append(new_token)
            # sklearn.feature_extraction 변환을 위해 단어들을 결합
            for i in range(len(facts_lst)):
                facts_lst[i] = ' '.join(facts_lst[i])
        else:
            print('컬럼이름을 변경하지 말아주세용')
    return first_party_lst, second_party_lst, facts_lst

#전처리함수 2(벡터화)
def preprocessing_2(first, second, facts, vec, vec_facts, train=True):
    if train:
        vec.fit(first + second)
        vec_facts.fit(facts)
    
    X1 = vec.transform(first).toarray()
    X2 = vec.transform(second).toarray()
    X3 = vec_facts.transform(facts).toarray()
    
    
    return np.concatenate([X1,X2,X3], axis=1)
        

In [11]:
# 문자열 전처리 1
cols = ['first_party', 'second_party', 'facts']
shortword = re.compile(r'\W*\b\w{1}\b')
tokenizer = TreebankWordTokenizer()
stopword = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

first_train, second_train, facts_train = preprocessing(train, cols, shortword, tokenizer, stopword, lemmatizer)
first_test, second_test, facts_test = preprocessing(test, cols, shortword, tokenizer, stopword, lemmatizer)

# 문자열 전처리 2(벡터화)
vec = CountVectorizer(ngram_range=(1,2)) #단어들의 출현 빈도로 문서 벡터화(모두 소문자로 변환) : 카운트 값이 높을 수록 중요한 단어로 인식
vec_facts = TfidfVectorizer(ngram_range=(1,2)) # 개별 문서에 자주 등장하는 단어에 높은 가중치를 주되, 모든 문서에 전반적으로 자주 나오는 단어의 경우 페널티 부여

X_train = preprocessing_2(first_train, second_train, facts_train, vec, vec_facts)
y_train = train['first_party_winner']
X_test = preprocessing_2(first_test, second_test, facts_test, vec, vec_facts, train=False)

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_33548\1584364167.py:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[col] = df[col].str.replace('.','') #.을 모두 제거함으로써 특정 의미 반영이 힘들 수 있을 듯


In [12]:
# 준비된 데이터 shape 확인

In [14]:
print('<train 데이터>')
print(X_train.shape, y_train.shape)
print()
print('<test 데이터>')
print(X_test.shape)

<train 데이터>
(2478, 196474) (2478,)

<test 데이터>
(1240, 196474)


In [15]:
#언더샘플링 : Neighbourhood Cleaning rule( CondensedNearestNEighbour과 EditedNearesNeighbours를 섞은 방법)
print(y_train.value_counts())
X_nc, y_nc = NeighbourhoodCleaningRule(n_neighbors=3).fit_resample(X_train, y_train)
print('Train Data Shape after UnderSampling')
print(X_nc.shape, y_nc.shape)
print('='*20)
print('Train target after UnderSampling')
print(y_nc.value_counts())

1    1649
0     829
Name: first_party_winner, dtype: int64
Train Data Shape after UnderSampling
(1452, 196474) (1452,)
Train target after UnderSampling
0    829
1    623
Name: first_party_winner, dtype: int64
